In [1]:
# Imports
import pandas as pd
from datetime import datetime
from datetime import timedelta

In [2]:
# Data

#Parameters
BloodTypes = {"O+", "O-", "A+", "A-", "B+", "B-", "AB+", "AB-"}
Gender = {"Male", "Female", "N/A"}
BloodComponents = {"RBC", "Platelets", "Plasma"}

#Eligibility
df = pd.read_csv("BloodDonationTypeWaitingPeriodByGender.csv")
LastBloodDonationRestrictions = df.set_index("Donation Type", drop = True)
HighRiskTravelCountries = pd.read_csv("HighRiskTravelCountries.csv")
df = pd.read_csv("ProcedureWaitingPeriod.csv")
ProceduralRestrictions = df.set_index("Procedure", drop = True)
noRiskTravelWaitPeriod = 21
highRiskTravelWaitPeriod = 365
bloodProcessingTime = 1

#Daily hospital demand 
df = pd.read_csv("SeasonalDemandRatesByBloodType.csv")
SeasonalDailyDemand = df.set_index("Season", drop = True)

#From database
donors = list()
appointments = list()

In [3]:
# This class represents a user of the application
class Donor:
    def __init__(self, donorId, gender, age, bloodType):
        self.donorId = donorId
        self.gender = gender
        self.age = age
        self.bloodType = bloodType
        self.appointments = list()

In [4]:
# This class represents a user's appointment 
class Appointment:
    def __init__(self, donor, bloodCompToDonate):
        self.donor = donor
        self.bloodCompToDonate = bloodCompToDonate
    
    def setAppointmentRestrictions(self, restrictions):
        self.restrictions = restrictions
        
    def setEligibleTime(self, eligibleTime):
        self.eligibleTime = eligibleTime
        
    def setDate(self, date):
        self.date = date
        
    def setWasSuccessful(self, wasSucessful):
        self.wasSuccessful = wasSuccessful
    
    def setIsCompleted(self, isCompleted):
        self.isCompleted = isCompleted

In [5]:
# Retrieving user input from UI (TODO - Henry)
def getBloodCompToDonate(userId):
    return "O-"

def getAppointmentRestrictions(userId):
    return {} # Dictionary mapping eligibility criteria -> date

In [6]:
import random
def getProbabilityOfSuccessfulDonation(donor):
    success = 0
    failed = 0
    for appointment in donor.appointments:
        if appointment.IsCompleted:
            if appointment.WasSuccessful:
                success += 1
            else:
                failed += 1
            
    if success == 0 and failed == 0: # default (research based)
        return 0.925
    
    else:
        (success)/(success + failed)

def getProcessingDays():
    rand = random.uniform(0,1)
    if rand <= 0.5:
        return 1
    else:
        return 0   

In [7]:
# For every user trying to book an appointment (our added feature)
donor = Donor(905, "Female", 24, "O+") # Represents logged in donor

# Creating/setting properties of new appointment
newAppointment = Appointment(donor, getBloodCompToDonate(donor.donorId))
restrictions = getAppointmentRestrictions(donor.donorId)
newAppointment.setAppointmentRestrictions(restrictions)

# Set this appointment's eligible start date as max of all time-based eligibility violations
eligibleDate = datetime.now()
for restriction in newAppointment.restrictions.keys():
    if restriction in str(LastBloodDonationRestrictions["Donation Type"]):
        donorGender = donor.Gender
        if donor.Gender is "N/A":
            donorGender = "Female" # if non-binary, set as female since longer waiting time
        eligibleDate = max(eligibleDate, newAppointment[restriction] + timedelta(days=int(LastBloodDonationRestrictions.loc[restriction, donorGender])))
    elif restriction in str(ProceduralRestrictions["Procedure"]):
        eligibleDate = max(eligibleDate, newAppointment[restriction] + int(ProceduralRestrictions.loc[restriction]))
    elif restriction in str(HighRiskTravelCountries['Country']):
        eligibleDate = max(eligibleDate, newAppointment[restriction] + timedelta(days=noRiskTravelWaitPeriod))
    else:
        eligibleDate = max(eligibleDate, newAppointment[restriction] + timedelta(days=noRiskTravelWaitPeriod))  
newAppointment.setEligibleTime(eligibleDate)

donor.appointments.append(newAppointment)
appointments.append(newAppointment)

In [ ]:
# Greedy heuristic of reccommending dates
import numpy as np
import operator
dateOptions = list()

# Determine how many donations we will theoretically have
dateDeviations = {}
for i in range(31):
    date = newAppointment.eligibleTime + timedelta(days = i)
    season = "Winter"
    if date.month >= 3 and date.month < 9:
        season = "Summer"
    bloodCompType = newAppointment.bloodCompToDonate
    if newAppointment.bloodCompToDonate == "Whole Blood":
        bloodCompType = newAppointment.donor.bloodType
    dateDeviations.update({date : SeasonalDailyDemand.loc[season,bloodCompType]})  
    for appointment in appointments:
        if len(bloodCompType) > 3 and appointment.bloodCompToDonate != newAppointment.bloodCompToDonate:
            continue
        elif len(bloodCompType) <= 3 and appointment.donor.bloodType != newAppointment.donor.bloodType:
            continue
        if appointment.date + timedelta(days = getProcessingDays()) == date:
            dateDeviations[date] -= np.random.binomial(1, getProbabilityOfSuccessfulDonation(appointment.donor))

sortedDateOptions = sorted(dateDeviations.items(), key=operator.itemgetter(1), reverse = True)